In [ ]:
!git clone https://github.com/WongKinYiu/yolov7.git

In [13]:
import wget

url = "https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt"
wget.download(url)

  1% [                                                                        ]   851968 / 75587165

  2% [.                                                                       ]  1982464 / 75587165

  3% [..                                                                      ]  2703360 / 75587165

  4% [...                                                                     ]  3375104 / 75587165

100% [........................................................................] 75587165 / 75587165

'yolov7.pt'

In [14]:
import sys
sys.path.append('./yolov7')

In [16]:
import torch

from models.yolo import Model
from utils.torch_utils import select_device
from PIL import Image
import cv2
import time


model = torch.load('yolov7.pt', map_location=torch.device('cpu'))
if isinstance(model, dict):
    model = model['ema' if model.get('ema') else 'model']  # load model

hub_model = Model(model.yaml).to(next(model.parameters()).device)  # create
hub_model.load_state_dict(model.float().state_dict())  # load state_dict
hub_model.names = model.names  # class names
hub_model = hub_model.autoshape()  # for file/URI/PIL/cv2/np inputs and NMS
device = select_device('0' if torch.cuda.is_available() else 'cpu')  # default to GPU if available
hub_model = hub_model.to(device)

img_dir = 'image/image.jpg'
img = cv2.imread(img_dir)
pil_image=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

results = hub_model([pil_image])
st = time.time()
results = hub_model([pil_image])
print(time.time() - st)
results = results.tolist()[0].pred

for result in results:
    x1, y1, x2, y2, conf, cls = result.cpu().tolist()
    img = cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 1)

cv2.imwrite('result/result_yolov7.png', img)


Adding autoShape... 
0.17904090881347656


True